# Generative AI and RAG

In [ ]:
'''
Title: Generative AI and RAG
Name: Irene Gichana
Date: 1 August 2025

'''
# Import Libraries
!pip install langchain langchain-community transformers sentence-transformers faiss-cpu pypdf

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

## Upload Document


In [ ]:
# Load PDF
loader = PyPDFLoader("CV.pdf")
docs = loader.load()

## Splitting the document into chunks
Used RecursiveCharacterTextSplitter with a sensible chunk size and overlap.

In [ ]:
# Split Documents into Chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

## Creating embeddings
Used HuggingFaceEmbeddings and FAISS to store vectors.

In [ ]:
# Create Embeddings and Vector Store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()


/tmp/ipython-input-1330068535.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## Adding Large Language Model(LLM)
Integrated google/flan-t5-large via Hugging Face Transformers.

In [ ]:
# LLM
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
flan_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


## Querying using RAG
Implemented a retrieval-augmented generation pipeline with context-based prompting.

In [ ]:
# RAG
def query_rag(question):
    relevant_docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    prompt = f"Answer the question using only the context:\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    response = flan_pipeline(
        prompt,
        max_new_tokens=200,
        temperature=0.9,      # Creativity control (lower = deterministic, higher = more diverse)
        top_k=50,             # Only sample from the top-k most likely tokens
        top_p=0.9,            # Nucleus sampling: only sample from tokens with cumulative prob <= top_p
        do_sample=True        # Enables sampling (required for temperature/top-k/top-p to work)
    )
    return response[0]['generated_text']

print(query_rag("Summarize the key points of this document in a paragraph of 200 words."))
print(query_rag("Summarize my CV in 3 sentences."))
print(query_rag("What are my main skills and competencies?"))
print(query_rag("List the programming languages I know."))
print(query_rag("What job roles have I held?"))
print(query_rag("Generate a professional bio based on my CV."))
print(query_rag("What are my top achievements?"))


/tmp/ipython-input-2190412791.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(question)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Skilled in analyzing large datasets, identifying trends, and providing actionable insights to support business decision-making. Eager to contribute analytical and problem-solving skills to dynamic, data-driven teams. Skilled in analyzing large datasets, identifying trends, and providing actionable insights to support business decision-making. Eager to contribute analytical and problem-solving skills to dynamic, data-driven teams. Skilled in analyzing large datasets, identifying trends, and providing actionable insights to support business decision-making. Eager to contribute analytical and problem-solving skills to dynamic, data-driven teams. Skilled in analyzing large datasets, identifying trends, and providing actionable insights to support business decision-making. Eager to contribute analytical and problem-solving skills to dynamic, data-driven teams. Skilled in analyzing large datasets, identifying trends, and providing actionable insights to


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


IRENE GICHANA Data Analyst Nairobi, Kenya |+254 707 301 767 | irenegichana5@gmail.com | LinkedIn | GitHub Professional Summary Motivated and detail-oriented Data Analyst with two years of experience in data analysis, reporting, and visualization. With a Bachelor of Science in Computer Science, I am proficient in SQL, Python, Power BI, and Excel, and have hands-on experience in data analysis, data integration, and KPI tracking. Skilled in analyzing large datasets, identifying trends, and providing actionable insights to support business decision-making. Eager to contribute analytical and problem-solving skills to dynamic, data-driven teams.


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

Data Analysis & Reporting: SQL, Python, Excel  Data Visualization: Power BI, Tableau  ETL & Data Integration: Data modeling, warehousing, ETL processes  Database Management: PostgreSQL, MySQL, Oracle  Database Management: PostgreSQL, MySQL, Oracle  KPI Analytics: Performance tracking, dashboards  Soft Skills: Attention to detail, communication, problem-solving, analytical skills Projects HR Analytics Dashboard – Link  Built a dynamic dashboard tracking KPIs such as employee attrition and employee satisfaction.  Helped identify why employees were turning over and Implemented targeted retention strategies that reduced attrition intent by 10%. Motorcycle Part Sales Analysis – Link  Analyzed 3-month transactional data using optimized SQL queries.  Delivered insights that enhanced inventory planning and reduced stockouts. Work Experience Sales


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


IRENE GICHANA Data Analyst Nairobi, Kenya |+254 707 301 767 | irenegichana5@gmail.com | LinkedIn | GitHub Professional Summary Motivated and detail-oriented Data Analyst with two years of experience in data analysis, reporting, and visualization. With a Bachelor of Science in Computer Science, I am proficient in SQL, Python, Power BI, and Excel, and have hands-on experience in data analysis, data April 2022 – June 2022  Provided IT support and resolved user system issues.  Assisted in data backup, software updates, and database maintenance. Education Egerton University Bachelor of Science in Computer Science – February 2023 Second Class Honors Certifications  Analyzing and Managing Data with Excel  Data Analysis with SQL integration, and KPI tracking. Skilled in analyzing large datasets, identifying trends, and providing actionable insights to support


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Database Management: PostgreSQL, MySQL, Oracle  KPI Analytics: Performance tracking, dashboards
